In [1]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# from google.colab import files
# uploaded = files.upload()
from google.colab import drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv('/content/drive/My Drive/[[[[ECCE 2025 CUET]]]]/Implementation of Existing Model/online_shoppers_intention.csv')

# Define numerical and categorical columns
numerical_columns = ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration',
                     'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']
categorical_columns = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend']

# Separate features and target variable
X = data[numerical_columns + categorical_columns]
y = data['Revenue']  # Assuming 'Revenue' is the dependent variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Mounted at /content/drive


**Preprocess the data**

In [2]:
# Preprocess the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(drop='first'), categorical_columns)
    ]
)

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

**Train the Linear Regression model and make prediction**

In [7]:
# Train the SVM model with a rbf kernel
#model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
#model.fit(X_train_transformed, y_train)

# Train the SVM model with a linear kernel
model = SVC(kernel='linear', C=1.0, random_state=42)
model.fit(X_train_transformed, y_train)

# Make predictions
y_pred = model.predict(X_test_transformed)

**Evaluate the model**

In [8]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(report)

Accuracy: 0.8702

Confusion Matrix:
[[1998   57]
 [ 263  148]]

Classification Report:
              precision    recall  f1-score   support

       False       0.88      0.97      0.93      2055
        True       0.72      0.36      0.48       411

    accuracy                           0.87      2466
   macro avg       0.80      0.67      0.70      2466
weighted avg       0.86      0.87      0.85      2466



**Feature Importance (Coefficients)**

In [12]:
# Feature Importance (Coefficients)
feature_names = (
    numerical_columns +
    preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns).tolist()
)

# Convert the sparse matrix to a dense array and flatten it
coefficients_dense = model.coef_[0].toarray()
coefficients_dense = coefficients_dense.flatten() # Flatten the array to 1-D

coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients_dense
}).sort_values(by='Coefficient', ascending=False)

print("\nFeature Importance:")
print(coefficients)


Feature Importance:
               Feature  Coefficient
8           PageValues     0.877447
60      TrafficType_16     0.143114
37          Browser_13     0.131789
36          Browser_12     0.119274
24  OperatingSystems_7     0.050083
..                 ...          ...
10           Month_Dec    -0.103126
15           Month_May    -0.132149
30           Browser_6    -0.210067
23  OperatingSystems_6    -0.316761
11           Month_Feb    -0.853573

[68 rows x 2 columns]


In [ ]:
# Make predictions and convert to binary using a threshold (e.g., 0.5)
y_pred = model.predict(X_test_transformed)
y_pred_binary = np.where(y_pred > 0.5, 1, 0)  # Convert to 0 or 1 based on threshold

# Calculate classification metrics using the binary predictions
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.86
Precision: 0.79
Recall: 0.24
F1 Score: 0.37
